In [1]:
import numpy as np
from evaldet.det.coco import calculate_pr_curve, calculate_pr_curve_np
from evaldet.dist import iou_dist

import numba
from numba import types

from tqdm import tqdm

In [2]:
N_FRAMES = 10000
N_P_FRAME = 100

preds_bbox = np.random.rand(N_FRAMES * N_P_FRAME, 4).astype(np.float32)
gts_bbox = np.random.rand(N_FRAMES * N_P_FRAME, 4).astype(np.float32)
preds_conf = np.random.rand(N_FRAMES * N_P_FRAME).astype(np.float32)

fp = numba.typed.Dict.empty(
    key_type=numba.int32,
    value_type=types.Tuple((types.int32, types.int32))
)
fg = numba.typed.Dict.empty(
    key_type=numba.int32,
    value_type=types.Tuple((types.int32, types.int32))
)
ious = numba.typed.Dict.empty(
    key_type=numba.int32,
    value_type=numba.float32[:, ::1]
)

ious_np = {}
fp_np = {}
fg_np = {}

for f in tqdm(range(N_FRAMES)):
    inds = (f * N_P_FRAME, (f+1) * N_P_FRAME)
    fp[f] =inds
    fg[f] = inds
    ious[f] = 1 - iou_dist(preds_bbox[inds[0]:inds[1]], gts_bbox[inds[0]:inds[1]])

    fp_np[f] =inds
    fg_np[f] = inds
    ious_np[f] = 1 - iou_dist(preds_bbox[inds[0]:inds[1]], gts_bbox[inds[0]:inds[1]])

area_range = (0.5, 100)
iou_threshold = 0.5

  0%|          | 0/10000 [00:00<?, ?it/s]/home/tadej/micromamba/envs/EvalDeT/lib/python3.10/site-packages/numba/typed/typeddict.py:33: NumbaTypeSafetyWarning: unsafe cast from UniTuple(int64 x 2) to UniTuple(int32 x 2). Precision may be lost.
  d[key] = value
/home/tadej/micromamba/envs/EvalDeT/lib/python3.10/site-packages/numba/typed/typeddict.py:33: NumbaTypeSafetyWarning: unsafe cast from int64 to int32. Precision may be lost.
  d[key] = value
100%|██████████| 10000/10000 [00:09<00:00, 1044.45it/s]


In [3]:
calculate_pr_curve(
    preds_bbox, gts_bbox, ious, preds_conf, fp, fg, area_range, iou_threshold
)

array([[0.0000000e+00, 4.9999997e-01, 3.3333331e-01, ..., 7.0667404e-01,
        7.0666993e-01, 7.0666581e-01],
       [0.0000000e+00, 6.5234976e-06, 6.5234976e-06, ..., 7.9088926e-01,
        7.9088926e-01, 7.9088926e-01]], dtype=float32)

In [4]:
%%timeit
calculate_pr_curve(
    preds_bbox, gts_bbox, ious, preds_conf, fp, fg, area_range, iou_threshold
)

527 ms ± 23.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
calculate_pr_curve_np(
    preds_bbox, gts_bbox, ious_np, preds_conf, fp_np, fg_np, area_range, iou_threshold
)

array([[0.0000000e+00, 5.0000000e-01, 3.3333334e-01, ..., 7.0667404e-01,
        7.0666993e-01, 7.0666581e-01],
       [0.0000000e+00, 6.5234976e-06, 6.5234976e-06, ..., 7.9088926e-01,
        7.9088926e-01, 7.9088926e-01]], dtype=float32)

In [6]:
%%timeit
calculate_pr_curve_np(
    preds_bbox, gts_bbox, ious_np, preds_conf, fp_np, fg_np, area_range, iou_threshold
)

643 ms ± 70.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
